In [1]:
#!pip install lightgbm
#импорт всех необходимых библиотек и инструментов
import numpy as np
import pandas as pd
#from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

#проверка скаченных библиотек и инструментов
import pkg_resources
packages = ['numpy', 'pandas', 'lightgbm', 'scikit-learn']
for package in packages:
    try:
        version = pkg_resources.get_distribution(package).version
        print(f"{package}: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"{package} is not installed")

numpy: 1.26.4
pandas: 2.1.4
lightgbm: 4.3.0
scikit-learn: 1.5.0


Загрузка данных

In [2]:
train_df = pd.read_parquet("train_data.pqt")
test_df = pd.read_parquet("test_data.pqt")

#одновременный вывод двух таблиц
from IPython.display import display
display(train_df.head(10)), display(test_df.head(10))

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other}
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other}
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.946458,0.442244,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other}
3,1,month_1,-0.081586,-0.091860,-0.114040,-0.080890,channel_code_2,city_14,city_type_0,None,...,0.945281,0.407762,0.369318,0.567093,0.785465,-0.184002,0.253523,0.462452,{other},{other}
4,1,month_2,-0.094962,-0.100504,-0.119302,-0.094307,channel_code_2,city_14,city_type_0,None,...,0.946066,0.430750,0.067275,0.559928,0.696576,-0.183854,0.255545,0.495419,{other},{other}
5,1,month_3,-0.090605,-0.114275,-0.114119,-0.089937,channel_code_2,city_14,city_type_0,None,...,0.948027,0.488221,0.043221,0.560788,0.707687,-0.167905,0.259011,0.605309,{other},{other}
6,2,month_1,-0.154685,-0.186795,-0.122805,-0.154215,channel_code_12,city_613,city_type_306,None,...,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.178674,0.252657,0.440474,{α},{α}
7,2,month_2,-0.152784,-0.193686,-0.122805,-0.152308,channel_code_12,city_613,city_type_306,None,...,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.177854,0.252657,0.440474,{α},{α}
8,2,month_3,-0.148737,-0.187003,-0.112416,-0.148249,channel_code_12,city_613,city_type_306,None,...,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.176302,0.252368,0.429485,{α},{α}
9,3,month_1,-0.156643,-0.204861,-0.125660,-0.156179,channel_code_14,city_21,city_type_0,index_city_code_46,...,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,{α},{α}


,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster
0,200000,month_4,-0.096224,0.335496,-0.125995,-0.095578,channel_code_12,city_14,city_type_0,None,...,0.010952,0.946066,0.407762,-0.153950,0.548895,0.541020,0.031742,0.257278,0.561353,{α}
1,200000,month_5,-0.024255,-0.059806,-0.124295,-0.023381,channel_code_12,city_14,city_type_0,None,...,0.006812,0.945281,0.396267,-0.150505,0.549468,0.552131,0.237817,0.264211,0.715199,{α}
2,200000,month_6,0.045988,0.049418,-0.125995,0.047079,channel_code_12,city_14,city_type_0,None,...,0.006812,0.945281,0.396267,-0.152800,0.549468,0.541020,0.387566,0.268543,0.836079,None
3,200001,month_4,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_76,city_type_0,None,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α}
4,200001,month_5,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_76,city_type_0,None,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α}
5,200001,month_6,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_76,city_type_0,None,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,None
6,200002,month_4,-0.096506,0.185905,-0.125995,-0.095856,channel_code_12,city_14,city_type_0,index_city_code_78,...,0.510730,0.950774,0.545693,0.715525,0.554913,0.718798,0.445811,0.254968,0.495419,{other}
7,200002,month_5,0.572242,1.502779,-0.125995,0.574963,channel_code_12,city_14,city_type_0,index_city_code_78,...,0.499912,0.949989,0.522704,2.442243,0.563940,0.841020,3.313686,0.256701,0.550364,{other}
8,200002,month_6,-0.048015,0.448252,-0.125995,-0.047215,channel_code_12,city_14,city_type_0,index_city_code_78,...,0.123154,0.946850,0.453739,2.614870,0.565087,0.818798,4.449125,0.258723,0.627287,None
9,200003,month_4,-0.156623,-0.204813,-0.125665,-0.156159,channel_code_7,city_31,city_type_0,None,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α}


(None, None)

Создадим для удобства обращения массивы категориальных и числовых признаков, также таргет признак

In [3]:
cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year", "end_cluster"
]
num_cols = train_df.select_dtypes(include=['number']).columns.tolist()
target_col = ['end_cluster']

Обозначение категориальных признаков

In [4]:
train_df[cat_cols] = train_df[cat_cols].astype("category")
cat_cols.remove("end_cluster")
test_df[cat_cols] = test_df[cat_cols].astype("category")
cat_cols.append("end_cluster")

Выведем статистику пропусков в данных

In [5]:
print(train_df.isnull().sum())
print()
#вывод всех возмождных вариантов каждого категориального признака
for col in cat_cols:
    print(f"{col} DISTRIBUTION")
    print(train_df[col].value_counts())
    print()

id                              0
date                            0
balance_amt_avg             65577
balance_amt_max             65577
balance_amt_min             65577
                            ...  
sum_cred_h_oper_3m              0
cnt_cred_h_oper_3m         165252
cnt_days_cred_h_oper_3m    165252
start_cluster                   0
end_cluster                     0
Length: 93, dtype: int64

channel_code DISTRIBUTION
channel_code
channel_code_8     102296
channel_code_12     79294
channel_code_21     56279
channel_code_0      52130
channel_code_14     31244
channel_code_1      31114
channel_code_9      30203
channel_code_5      26722
channel_code_18     24781
channel_code_6      23075
channel_code_4      21203
channel_code_2      20285
channel_code_19     13029
channel_code_7      12294
channel_code_10      9863
channel_code_16      6549
channel_code_20      5569
channel_code_3       2599
channel_code_15      2010
channel_code_17      1424
channel_code_11      1242
channel_code_24

Заполним пропуски в поле id на нули, и пропуски среди числовых признаков на медиальные значения

In [6]:
#замена всех пропусков у признака id на 0
train_df['id'] = train_df['id'].fillna(0)
test_df['id'] = test_df['id'].fillna(0)
#замена всех пропусков у числовых признаков на медиальные значения
for feature in num_cols:
    train_df[feature].fillna(train_df[feature].median(), inplace=True)
    test_df[feature].fillna(test_df[feature].median(), inplace=True)
train_df.head(4)

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other}
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other}
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.946458,0.442244,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other}
3,1,month_1,-0.081586,-0.091860,-0.114040,-0.080890,channel_code_2,city_14,city_type_0,NaN,...,0.945281,0.407762,0.369318,0.567093,0.785465,-0.184002,0.253523,0.462452,{other},{other}


Удалим те признаки, которые мало влияют на таргет переменную

In [7]:
train_df = train_df.drop(columns=['index_city_code',  'okved', 'ogrn_month', 'ogrn_year', 'ogrn_days_end_quarter', 'max_founderpres', 'min_founderpres', 'ogrn_exist_months'])
test_df = test_df.drop(columns=['index_city_code', 'okved', 'ogrn_month', 'ogrn_year', 'ogrn_days_end_quarter', 'max_founderpres', 'min_founderpres', 'ogrn_exist_months'])
cat_cols = [
    "channel_code",  "segment", "start_cluster",  "end_cluster"
]
train_df = train_df.dropna()
train_df.shape

(509415, 85)

Создадим на основе нескольких старых признаков новые

In [8]:
#признак, объединение количества всех входящих операций
train_df['cnt'] = train_df['cnt_a_oper_1m'] * train_df['cnt_b_oper_1m'] * train_df['cnt_c_oper_1m'] * train_df['cnt_cred_d_oper_1m'] * train_df['cnt_cred_e_oper_1m'] * train_df['cnt_cred_f_oper_1m'] * train_df['cnt_days_deb_g_oper_1m'] * train_df['cnt_days_cred_h_oper_1m']
test_df['cnt'] = test_df['cnt_a_oper_1m'] * test_df['cnt_b_oper_1m'] * test_df['cnt_c_oper_1m'] * test_df['cnt_cred_d_oper_1m'] * test_df['cnt_cred_e_oper_1m'] * test_df['cnt_cred_f_oper_1m'] * test_df['cnt_days_deb_g_oper_1m'] * test_df['cnt_days_cred_h_oper_1m']
#признак, объединение всех данных по балансу клиента
train_df['ost'] = train_df['balance_amt_avg'] + train_df['balance_amt_max'] + train_df['balance_amt_min']  + train_df['balance_amt_day_avg']
test_df['ost'] = test_df['balance_amt_avg'] + test_df['balance_amt_max'] + test_df['balance_amt_min']  + test_df['balance_amt_day_avg']
#признак, диапозон баланса клиента
train_df['dia'] = train_df['balance_amt_max'] - train_df['balance_amt_min']
test_df['dia'] = test_df['balance_amt_max'] - test_df['balance_amt_min']
#признак, объединение города и его типа
train_df['cit'] = train_df['city'].astype(str) + "_" +  train_df['city_type'].astype(str)
train_df['cit'] = train_df['cit'].astype("category")
test_df['cit'] = test_df['city'].astype(str) + "_" +  test_df['city_type'].astype(str)
test_df['cit'] = test_df['cit'].astype("category")
cat_cols.append('cit')

#удаление некоторых признаков которые являются частью ново создланных признаков
train_df = train_df.drop(columns=['balance_amt_avg', 'balance_amt_max', 'balance_amt_min', 'balance_amt_day_avg', 'city', 'city_type'])
test_df = test_df.drop(columns=['balance_amt_avg', 'balance_amt_max', 'balance_amt_min', 'balance_amt_day_avg', 'city', 'city_type'])
train_df.head(5)

,id,date,channel_code,ogrn_days_end_month,ft_registration_date,segment,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,...,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster,cnt,ost,dia,cit
0,0,month_1,channel_code_5,-0.488553,2.452182,segment_1,0.942275,0.536013,0.511490,-0.188835,...,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other},0.001682,3.485645,-0.581715,city_23_city_type_0
1,0,month_2,channel_code_5,-0.488553,2.469917,segment_1,0.645704,0.536378,0.486425,-0.188835,...,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other},0.001436,5.393935,-1.626694,city_23_city_type_0
2,0,month_3,channel_code_5,-0.488553,2.485936,segment_1,0.403604,0.613167,0.480547,-0.188835,...,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other},0.001682,2.558695,0.310211,city_23_city_type_0
3,1,month_1,channel_code_2,0.324343,0.174035,segment_1,0.014051,0.043800,0.052041,-0.188835,...,0.785465,-0.184002,0.253523,0.462452,{other},{other},0.000214,-0.368377,0.022180,city_14_city_type_0
4,1,month_2,channel_code_2,0.324343,0.191770,segment_1,-0.057593,0.035027,0.033554,-0.188835,...,0.696576,-0.183854,0.255545,0.495419,{other},{other},0.000927,-0.409074,0.018798,city_14_city_type_0


Импорт модели, деление тренировочных данных на обучающую и проверочную выборки

In [9]:
#!pip install catboost

#импорт классификатора
from catboost import CatBoostClassifier

#деление данных
X = train_df.drop(["id", "date", "end_cluster"], axis=1)
y = train_df["end_cluster"]

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Обучение модели и проверка точности по метрике ROC AUC

In [10]:
if 'end_cluster' in cat_cols:
    cat_cols.remove('end_cluster')

#настройки самой модели
model = CatBoostClassifier(iterations=100, 
                           learning_rate=0.1, 
                           depth=4, 
                           cat_features=cat_cols, 
                           verbose=20)

model.fit(x_train, y_train)
#y_pred = model.predict_proba(x_val)

#метрика для оценивания
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

cluster_weights = pd.read_excel("cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

y_pred_proba = model.predict_proba(x_val)
roc_auc_score_weighted = weighted_roc_auc(y_val, y_pred_proba, model.classes_, weights_dict)
print(f"Weighted ROC AUC Score: {roc_auc_score_weighted}")

0:	learn: 2.0458284	total: 2.39s	remaining: 3m 56s
20:	learn: 0.9629111	total: 47.7s	remaining: 2m 59s
40:	learn: 0.8868115	total: 1m 31s	remaining: 2m 11s
60:	learn: 0.8636772	total: 2m 15s	remaining: 1m 26s
80:	learn: 0.8503699	total: 2m 57s	remaining: 41.7s
99:	learn: 0.8419589	total: 3m 38s	remaining: 0us
Weighted ROC AUC Score: 0.9008691805875031


Обработка признака start_cluster в тестовых данных

In [12]:
test_df[cat_cols] = test_df[cat_cols].astype("str")
test_df["start_cluster"] = train_df["start_cluster"].mode()[0]
test_df["start_cluster"] = test_df["start_cluster"].astype("category")
sample_submission_df = pd.read_csv("sample_submission.csv")
print(sample_submission_df.shape)
sample_submission_df.head()

(100000, 18)


,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
1,200001,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
2,200002,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
3,200003,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
4,200004,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05


In [13]:
test_df.head(6)

,id,date,channel_code,ogrn_days_end_month,ft_registration_date,segment,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,...,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,cnt,ost,dia,cit
0,200000,month_4,channel_code_12,-1.533705,-0.653810,segment_3,0.416833,0.332409,0.676573,0.532156,...,0.548895,0.541020,0.031742,0.257278,0.561353,{α},0.001388,0.017699,0.461491,city_14_city_type_0
1,200000,month_5,channel_code_12,-1.533705,-0.636647,segment_3,0.433195,0.284317,0.688449,-0.069498,...,0.549468,0.552131,0.237817,0.264211,0.715199,{α},0.001447,-0.231736,0.064489,city_14_city_type_0
2,200000,month_6,channel_code_12,-1.533705,-0.618911,segment_3,0.223961,0.285376,0.671862,2.698111,...,0.549468,0.541020,0.387566,0.268543,0.836079,{α},0.001339,0.016490,0.175413,city_14_city_type_0
3,200001,month_4,channel_code_9,0.092087,-0.662964,segment_3,-0.241963,-0.255143,-0.251583,-0.188835,...,0.547176,0.429909,-0.201123,0.251501,0.396518,{α},0.000214,-0.643756,-0.079065,city_76_city_type_0
4,200001,month_5,channel_code_9,0.092087,-0.645800,segment_3,-0.241963,-0.255143,-0.251583,-0.188835,...,0.547176,0.429909,-0.201123,0.251501,0.396518,{α},0.000214,-0.643756,-0.079065,city_76_city_type_0
5,200001,month_6,channel_code_9,0.092087,-0.628065,segment_3,-0.241963,-0.255143,-0.251583,-0.188835,...,0.547176,0.429909,-0.201123,0.251501,0.396518,{α},0.000214,-0.643756,-0.079065,city_76_city_type_0


Для тестовой выборки будем использовать только последний месяц

In [14]:
last_m_test_df = test_df[test_df["date"] == "month_6"]
last_m_test_df = last_m_test_df.drop(["id", "date"], axis=1)
last_m_test_df.head(5)

,channel_code,ogrn_days_end_month,ft_registration_date,segment,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,cnt_a_oper_1m,sum_b_oper_1m,...,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,cnt,ost,dia,cit
2,channel_code_12,-1.533705,-0.618911,segment_3,0.223961,0.285376,0.671862,2.698111,0.431041,-0.030225,...,0.549468,0.541020,0.387566,0.268543,0.836079,{α},0.001339,0.016490,0.175413,city_14_city_type_0
5,channel_code_9,0.092087,-0.628065,segment_3,-0.241963,-0.255143,-0.251583,-0.188835,0.417022,-0.030225,...,0.547176,0.429909,-0.201123,0.251501,0.396518,{α},0.000214,-0.643756,-0.079065,city_76_city_type_0
8,channel_code_12,-1.069193,-0.721319,segment_3,4.142519,1.969559,1.211625,13.082660,0.529172,-0.030225,...,0.565087,0.818798,4.449125,0.258723,0.627287,{α},0.001523,0.227027,0.574247,city_14_city_type_0
11,channel_code_7,-0.256297,-0.873501,segment_3,-0.241963,-0.255143,-0.251583,-0.188835,0.417022,-0.030225,...,0.547176,0.429909,-0.201123,0.251501,0.396518,{α},0.000214,-0.643007,-0.079312,city_31_city_type_0
13,channel_code_7,0.672727,0.732416,segment_3,-0.273794,-0.294571,-0.296092,-0.188835,0.417022,-0.030225,...,0.547032,0.418798,-0.201123,0.250924,0.374540,{α},0.000124,-0.635461,-0.075937,nan_nan


In [15]:
test_pred_proba = model.predict_proba(last_m_test_df)
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=model.classes_)
#упорядочевание данных в алфавитном порядке
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

Конвертирование предсказания в csv

In [16]:
sample_submission_df[sorted_classes] = test_pred_proba_df
sample_submission_df.to_csv("domino_submission.csv", index=False)